In [ ]:
from IPython.display import Image

## Seq2Seq 모델을 활용한 챗봇 생성

### Seq2Seq 모델의 개요

In [ ]:
Image('https://wikidocs.net/images/page/24996/%EC%9D%B8%EC%BD%94%EB%8D%94%EB%94%94%EC%BD%94%EB%8D%94%EB%AA%A8%EB%8D%B8.PNG')

## 데이터셋에 필요한 라이브러리를 다운로드 받습니다.

`Korpora`는 한글 자연어처리 데이터)셋입니다.

- [깃헙 주소 링크](https://github.com/ko-nlp/Korpora)
- [공식 도큐먼트](https://pypi.org/project/Korpora/)

설치 명령어

In [ ]:
# !pip install Korpora

- 이 중 챗봇용 데이터셋인 `KoreanChatbotKorpus`를 다운로드 받습니다.
- `KoreanChatbotKorpus` 데이터셋을 활용하여 챗봇 모델을 학습합니다.
- text, pair로 구성되어 있습니다.
- 질의는 **text**, 답변은 **pair**입니다.

In [ ]:
from Korpora import KoreanChatbotKorpus
corpus = KoreanChatbotKorpus()

예시 텍스트를 보면 구어체로 구성되어 있습니다.

In [ ]:
corpus.get_all_texts()[:10]

`get_all_pairs()`는 `text`와 `pair`가 쌍으로 이루어져 있습니다.

In [ ]:
corpus.get_all_pairs()[0].text

In [ ]:
corpus.get_all_pairs()[0].pair

## 데이터 전처리

**question**과 **answer**를 분리합니다.

**question**은 질의로 활용될 데이터셋, **answer**는 답변으로 활용될 데이터 셋입니다.

In [ ]:
texts = []
pairs = []

for sentence in corpus.get_all_pairs():
    texts.append(sentence.text)
    pairs.append(sentence.pair)

In [ ]:
list(zip(texts, pairs))[:5]

### 특수문자는 제거합니다.

**한글과 숫자를 제외한 특수문자를 제거**하도록 합니다.

*[참고] 튜토리얼에서는 특수문자와 영문자를 제거하나, 실제 프로젝트에 적용해보기 위해서는 신중히 결정해야합니다.*

*챗봇 대화에서 영어도 많이 사용되고, 특수문자도 굉장히 많이 사용됩니다. 따라서, 선택적으로 제거할 특수기호나 영문자를 정의한 후에 전처리를 진행하야합니다.*

In [ ]:
# re 모듈은 regex expression을 적용하기 위하여 활용합니다.
import re

In [ ]:
def clean_sentence(sentence):
    # 한글, 숫자를 제외한 모든 문자는 제거합니다.
    sentence = re.sub(r'[^0-9ㄱ-ㅎㅏ-ㅣ가-힣 ]',r'', sentence)
    return sentence

**적용한 예시**

한글, 숫자 이외의 모든 문자를 전부 제거됨을 확인할 수 있습니다.

In [ ]:
clean_sentence('12시 땡^^!??')

In [ ]:
clean_sentence('abcef가나다^^$%@12시 땡^^!??')

### 한글 형태소 분석기 (Konlpy)

### 형태소 분석기를 활용하여 문장을 분리합니다.

```가방에 들어가신다 -> 가방/NNG + 에/JKM + 들어가/VV + 시/EPH + ㄴ다/EFN```

- **형태소 분석** 이란 형태소를 비롯하여, 어근, 접두사/접미사, 품사(POS, part-of-speech) 등 다양한 언어적 속성의 구조를 파악하는 것입니다.
- **konlpy 형태소 분석기를 활용**하여 한글 문장에 대한 토큰화처리를 보다 효율적으로 처리합니다.



[공식 도큐먼트](https://konlpy-ko.readthedocs.io/ko/v0.4.3/morph/)

**설치**

In [ ]:
# !pip install konlpy

konlpy 내부에는 Kkma, Okt, Twitter 등등의 형태소 분석기가 존재하지만, 이번 튜토리얼에서는 Okt를 활용하도록 하겠습니다.

In [ ]:
from konlpy.tag import Okt

In [ ]:
okt = Okt()

In [ ]:
# 형태소 변환에 활용하는 함수
# morphs 함수 안에 변환한 한글 문장을 입력 합니다.
def process_morph(sentence):
    return ' '.join(okt.morphs(sentence))

**Seq2Seq** 모델이 학습하기 위한 데이터셋을 구성할 때, 다음과 같이 **3가지 데이터셋**을 구성합니다.

- `question`: encoder input 데이터셋 (질의 전체)
- `answer_input`: decoder input 데이터셋 (답변의 시작). START 토큰을 문장 처음에 추가 합니다.
- `answer_output`: decoder output 데이터셋 (답변의 끝). END 토큰을 문장 마지막에 추가 합니다.

In [ ]:
def clean_and_morph(sentence, is_question=True):
    # 한글 문장 전처리
    sentence = clean_sentence(sentence)
    # 형태소 변환
    sentence = process_morph(sentence)
    # Question 인 경우, Answer인 경우를 분기하여 처리합니다.
    if is_question:
        return sentence
    else:
        # START 토큰은 decoder input에 END 토큰은 decoder output에 추가합니다.
        return ('<START> ' + sentence, sentence + ' <END>')

In [ ]:
def preprocess(texts, pairs):
    questions = []
    answer_in = []
    answer_out = []

    # 질의에 대한 전처리
    for text in texts:
        # 전처리와 morph 수행
        question = clean_and_morph(text, is_question=True)
        questions.append(question)

    # 답변에 대한 전처리
    for pair in pairs:
        # 전처리와 morph 수행
        in_, out_ = clean_and_morph(pair, is_question=False)
        answer_in.append(in_)
        answer_out.append(out_)
    
    return questions, answer_in, answer_out

In [ ]:
questions, answer_in, answer_out = preprocess(texts, pairs)

In [ ]:
questions[:5]

In [ ]:
answer_in[:5]

In [ ]:
answer_out[:5]

In [ ]:
all_sentences = questions + answer_in + answer_out

In [ ]:
a = (' '.join(questions) + ' '.join(answer_in) + ' '.join(answer_out)).split()
len(set(a))

## 토큰화

In [ ]:
import numpy as np
import warnings
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# WARNING 무시
warnings.filterwarnings('ignore')

**토큰의 정의**

In [ ]:
tokenizer = Tokenizer(filters='', lower=False, oov_token='<OOV>')

**Tokenizer**로 문장에 대한 Word-Index Vocabulary(단어 사전)을 만듭니다.

In [ ]:
tokenizer.fit_on_texts(all_sentences)

**단어 사전 10개 출력**

In [ ]:
for word, idx in tokenizer.word_index.items():
    print(f'{word}\t\t => \t{idx}')
    if idx > 10:
        break

**토큰의 갯수 확인**

In [ ]:
len(tokenizer.word_index)

## 치환: 텍스트를 시퀀스로 인코딩 (`texts_to_sequences`)

In [ ]:
question_sequence = tokenizer.texts_to_sequences(questions)
answer_in_sequence = tokenizer.texts_to_sequences(answer_in)
answer_out_sequence = tokenizer.texts_to_sequences(answer_out)

## 문장의 길이 맞추기 (`pad_sequences`)

In [ ]:
MAX_LENGTH = 30

In [ ]:
question_padded = pad_sequences(question_sequence, maxlen=MAX_LENGTH, truncating='post', padding='post')
answer_in_padded = pad_sequences(answer_in_sequence, maxlen=MAX_LENGTH, truncating='post', padding='post')
answer_out_padded = pad_sequences(answer_out_sequence, maxlen=MAX_LENGTH, truncating='post', padding='post')

In [ ]:
question_padded.shape

In [ ]:
answer_in_padded.shape, answer_out_padded.shape

## 모델

In [ ]:
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint

## 학습용 인코더 (Encoder)

In [ ]:
class Encoder(tf.keras.Model):
    def __init__(self, units, vocab_size, embedding_dim, time_steps):
        super(Encoder, self).__init__()
        self.embedding = Embedding(vocab_size, embedding_dim, input_length=time_steps)
        self.dropout = Dropout(0.2)
        self.lstm = LSTM(units, return_state=True)
        
    def call(self, inputs):
        x = self.embedding(inputs)
        x = self.dropout(x)
        x, hidden_state, cell_state = self.lstm(x)
        return [hidden_state, cell_state]

## 학습용 디코더 (Decoder)

In [ ]:
class Decoder(tf.keras.Model):
    def __init__(self, units, vocab_size, embedding_dim, time_steps):
        super(Decoder, self).__init__()
        self.embedding = Embedding(vocab_size, embedding_dim, input_length=time_steps)
        self.dropout = Dropout(0.2)
        self.lstm = LSTM(units, 
                         return_state=True, 
                         return_sequences=True, 
                        )
        self.dense = Dense(vocab_size, activation='softmax')
    
    def call(self, inputs, initial_state):
        x = self.embedding(inputs)
        x = self.dropout(x)
        x, hidden_state, cell_state = self.lstm(x, initial_state=initial_state)        
        x = self.dense(x)
        return x, hidden_state, cell_state

## 모델 결합

In [39]:
class Seq2Seq(tf.keras.Model):
    def __init__(self, units, vocab_size, embedding_dim, time_steps, start_token, end_token):
        super(Seq2Seq, self).__init__()
        self.start_token = start_token
        self.end_token = end_token
        self.time_steps = time_steps
        
        self.encoder = Encoder(units, vocab_size, embedding_dim, time_steps)
        self.decoder = Decoder(units, vocab_size, embedding_dim, time_steps)
        
    def call(self, inputs, training=True):
        if training:
            encoder_inputs, decoder_inputs = inputs
            context_vector = self.encoder(encoder_inputs)
            decoder_outputs, _, _ = self.decoder(inputs=decoder_inputs, initial_state=context_vector)
            return decoder_outputs
        else:
            context_vector = self.encoder(inputs)
            target_seq = tf.constant([[self.start_token]], dtype=tf.float32)
            results = tf.TensorArray(tf.int32, self.time_steps)
            
            for i in tf.range(self.time_steps):
                decoder_output, decoder_hidden, decoder_cell = self.decoder(target_seq, initial_state=context_vector)
                decoder_output = tf.cast(tf.argmax(decoder_output, axis=-1), dtype=tf.int32)
                decoder_output = tf.reshape(decoder_output, shape=(1, 1))
                results = results.write(i, decoder_output)
                
                if decoder_output == self.end_token:
                    break
                    
                target_seq = decoder_output
                context_vector = [decoder_hidden, decoder_cell]
                
            return tf.reshape(results.stack(), shape=(1, self.time_steps))

## 단어별 원핫인코딩 적용

단어별 원핫인코딩을 적용하는 이유는 decoder의 output(출력)을 원핫인코딩 vector로 변환하기 위함

In [40]:
VOCAB_SIZE = len(tokenizer.word_index)+1

In [41]:
def convert_to_one_hot(padded):
    # 원핫인코딩 초기화
    one_hot_vector = np.zeros((len(answer_out_padded), MAX_LENGTH, VOCAB_SIZE))

    # 디코더 목표를 원핫인코딩으로 변환
    # 학습시 입력은 인덱스이지만, 출력은 원핫인코딩 형식임
    for i, sequence in enumerate(answer_out_padded):
        for j, index in enumerate(sequence):
            one_hot_vector[i, j, index] = 1

    return one_hot_vector

In [42]:
answer_in_one_hot = convert_to_one_hot(answer_in_padded)
answer_out_one_hot = convert_to_one_hot(answer_out_padded)

In [43]:
answer_in_one_hot[0].shape, answer_in_one_hot[0].shape

((30, 12638), (30, 12638))

## 변환된 index를 다시 단어로 변환

In [44]:
def convert_index_to_text(indexs, end_token): 
    
    sentence = ''
    
    # 모든 문장에 대해서 반복
    for index in indexs:
        if index == end_token:
            # 끝 단어이므로 예측 중비
            break;
        # 사전에 존재하는 단어의 경우 단어 추가
        if index > 0 and tokenizer.index_word[index] is not None:
            sentence += tokenizer.index_word[index]
        else:
        # 사전에 없는 인덱스면 빈 문자열 추가
            sentence += ''
            
        # 빈칸 추가
        sentence += ' '
    return sentence

## 학습 (Training)

**하이퍼 파라미터 정의**

In [45]:
BUFFER_SIZE = 1000
BATCH_SIZE = 16
EMBEDDING_DIM = 100
TIME_STEPS = MAX_LENGTH
START_TOKEN = tokenizer.word_index['<START>']
END_TOKEN = tokenizer.word_index['<END>']

UNITS = 128

VOCAB_SIZE = len(tokenizer.word_index)+1
DATA_LENGTH = len(questions)
SAMPLE_SIZE = 3
NUM_EPOCHS = 20

**체크포인트 생성**

In [46]:
checkpoint_path = 'model/seq2seq-chatbot-no-attention-checkpoint.ckpt'
checkpoint = ModelCheckpoint(filepath=checkpoint_path, 
                             save_weights_only=True,
                             save_best_only=True, 
                             monitor='loss', 
                             verbose=1
                            )

**분산환경 설정**

In [47]:
strategy = tf.distribute.MirroredStrategy()
FLAG = True
if strategy.num_replicas_in_sync  > 1 and FLAG:
    MULTIPLE_BATCH = strategy.num_replicas_in_sync
    print(f'분산환경 사용 >> GPU: {MULTIPLE_BATCH}')
else:
    print(f'분산환경 미사용')
    MULTIPLE_BATCH = 1

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
분산환경 사용 >> GPU: 2


**모델 생성 & compile**

In [48]:
# 분산 환경 적용시
if MULTIPLE_BATCH > 1:
    print(f'분산환경 사용 >> GPU: {MULTIPLE_BATCH}')
    with strategy.scope():
        seq2seq = Seq2Seq(UNITS, VOCAB_SIZE, EMBEDDING_DIM, TIME_STEPS, START_TOKEN, END_TOKEN)
        seq2seq.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
else:
    print(f'분산환경 미사용')
    seq2seq = Seq2Seq(UNITS, VOCAB_SIZE, EMBEDDING_DIM, TIME_STEPS, START_TOKEN, END_TOKEN)
    seq2seq.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

분산환경 사용 >> GPU: 2


In [49]:
# 연속하여 학습시 체크포인트를 로드하여 이어서 학습합니다.
# seq2seq.load_weights(checkpoint_path)

In [50]:
def make_prediction(model, question_inputs):
    results = model(inputs=question_inputs, training=False)
    # 변환된 인덱스를 문장으로 변환
    results = np.asarray(results).reshape(-1)
    return results

In [ ]:
for epoch in range(NUM_EPOCHS):
    print(f'processing epoch: {epoch * 10 + 1}...')
    seq2seq.fit([question_padded, answer_in_padded],
                answer_out_one_hot,
                epochs=10,
                batch_size=BATCH_SIZE*MULTIPLE_BATCH, 
                callbacks=[checkpoint]
               )
    # 랜덤한 샘플 번호 추출
    samples = np.random.randint(DATA_LENGTH, size=SAMPLE_SIZE)

    # 예측 성능 테스트
    for idx in samples:
        question_inputs = question_padded[idx]
        # 문장 예측
        results = make_prediction(seq2seq, np.expand_dims(question_inputs, 0))
        
        # 변환된 인덱스를 문장으로 변환
        results = convert_index_to_text(results, END_TOKEN)
        
        print(f'Q: {questions[idx]}')
        print(f'A: {results}\n')
        print()

processing epoch: 1...
Epoch 1/10
Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.
INFO:tensorflow:batch_all_reduce: 8 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:GPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1').
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:GPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1').
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:

370/370 [==============================] - 10s 28ms/step - loss: 0.7093 - acc: 0.8742
Epoch 9/10
368/370 [============================>.] - ETA: 0s - loss: 0.6920 - acc: 0.8761
Epoch 00009: loss improved from 0.70934 to 0.69189, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
370/370 [==============================] - 10s 28ms/step - loss: 0.6919 - acc: 0.8761
Epoch 10/10
369/370 [============================>.] - ETA: 0s - loss: 0.6751 - acc: 0.8784
Epoch 00010: loss improved from 0.69189 to 0.67522, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
370/370 [==============================] - 10s 28ms/step - loss: 0.6752 - acc: 0.8784
Q: 시험 공부 큰일
A: 마음 이 좀 더 무너져요 


Q: 아부 도 기술 인가 봐
A: 마음 이 좀 더 무너져요 


Q: 참아야 하나
A: 마음 이 좀 더 무너져요 


processing epoch: 21...
Epoch 1/10
370/370 [==============================] - ETA: 0s - loss: 0.6591 - acc: 0.8804
Epoch 00001: loss improved from 0.67522 to 0.65912, saving model to model/seq2seq-chatbot-no-attention-check

370/370 [==============================] - ETA: 0s - loss: 0.3407 - acc: 0.9334
Epoch 00001: loss improved from 0.34514 to 0.34066, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
370/370 [==============================] - 10s 27ms/step - loss: 0.3407 - acc: 0.9334
Epoch 2/10
368/370 [============================>.] - ETA: 0s - loss: 0.3371 - acc: 0.9340
Epoch 00002: loss improved from 0.34066 to 0.33714, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
370/370 [==============================] - 10s 27ms/step - loss: 0.3371 - acc: 0.9339
Epoch 3/10
369/370 [============================>.] - ETA: 0s - loss: 0.3335 - acc: 0.9348
Epoch 00003: loss improved from 0.33714 to 0.33353, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
370/370 [==============================] - 10s 27ms/step - loss: 0.3335 - acc: 0.9348
Epoch 4/10
368/370 [============================>.] - ETA: 0s - loss: 0.3297 - acc: 0.9351
Epoch 00004: loss improved from 

Epoch 8/10
368/370 [============================>.] - ETA: 0s - loss: 0.2453 - acc: 0.9478
Epoch 00008: loss improved from 0.24786 to 0.24514, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
370/370 [==============================] - 10s 28ms/step - loss: 0.2451 - acc: 0.9479
Epoch 9/10
369/370 [============================>.] - ETA: 0s - loss: 0.2409 - acc: 0.9484
Epoch 00009: loss improved from 0.24514 to 0.24096, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
370/370 [==============================] - 10s 28ms/step - loss: 0.2410 - acc: 0.9484
Epoch 10/10
370/370 [==============================] - ETA: 0s - loss: 0.2382 - acc: 0.9487
Epoch 00010: loss improved from 0.24096 to 0.23816, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
370/370 [==============================] - 10s 28ms/step - loss: 0.2382 - acc: 0.9487
Q: 짝남 잊으려고 나 혼자 나쁘게 생각 하는 내 자신 이 증오 스러워
A: 사랑 의 예의 가 없는 사람 이네 요 


Q: 난 또 바보 ㅠㅠ
A: 확신 이 들 때 까지 준비 해보세요 


Q: 난 

Q: 짝사랑 하는 사람과 친해질 수 있는 방법 조언 좀
A: 공통 관심사 를 찾아보세요 


Q: 짝남 한테 고백 한 다 안 한다
A: 무시 당하는 기분 이 들어서 너무 외 롭고 상처 받게 된다고 차분하고 부드럽게 말 해보세요 


processing epoch: 151...


## 예측

In [ ]:
# 자연어 (질문 입력) 대한 전처리 함수
def make_question(sentence):
    sentence = clean_and_morph(sentence)
    question_sequence = tokenizer.texts_to_sequences([sentence])
    question_padded = pad_sequences(question_sequence, maxlen=MAX_LENGTH, truncating='post', padding='post')
    return question_padded

In [ ]:
make_question('오늘 날씨가 정말 화창합니다')

In [ ]:
make_question('찐찐찐찐찐이야~ 완전 찐이야~')

In [ ]:
def run_chatbot(question):
    question_inputs = make_question(question)
    results = make_prediction(seq2seq, question_inputs)
    results = convert_index_to_text(results, END_TOKEN)
    return results

## 유저로부터 Text 입력 값을 받아 답변 출력

In [ ]:
while True:
    user_input = input('<< 말을 걸어 보세요!\n')
    if user_input == 'q':
        break
    print('>> 챗봇 응답: {}'.format(run_chatbot(user_input)))